In [1]:
# Robustes spaCy-Setup + Regelsystem
from __future__ import annotations
import re, json
from pathlib import Path

import spacy
import importlib.util
from spacy.matcher import Matcher, PhraseMatcher
from typing import Iterable, List, Dict, Set


def load_nlp_de():
    """
    Robustes Laden: zuerst Paket 'de_core_news_sm', dann spacy.load(),
    sonst Fallback auf ein leeres deutsches Pipeline-Objekt.
    """
    try:
        if importlib.util.find_spec("de_core_news_sm") is not None:
            import de_core_news_sm  # type: ignore
            return de_core_news_sm.load()
        return spacy.load("de_core_news_sm")
    except Exception:
        # Fallback: blankes Deutschen-Objekt (ohne Tagger/Parser, aber Tokenizer)
        return spacy.blank("de")


nlp = load_nlp_de()

# ---------- Phrase-Matcher für Audio-Adjektive ----------
ADJ_TERMS = [
    "boxig","mulmig","harsch","scharf","nasal","dull","weich","knackig","punchy",
    "spitz","dumpf","blechern","luftig","warm","brillant"
]
phr = PhraseMatcher(nlp.vocab, attr="LOWER")
phr.add("ADJ_AUDIO", [nlp.make_doc(t) for t in ADJ_TERMS])

# ---------- Token-Matcher ----------
matcher = Matcher(nlp.vocab)

# 1) Frequenzangaben (z. B. "800 Hz", "3 kHz", "1.2kHz", "800–1000 Hz")
#    Variante A: [Zahl] [optional -/–/— Zahl] [Einheit]
#    Variante B: "1.2kHz" / "800Hz" als einzelnes Token
matcher.add(
    "FREQ_HZ",
    [
        [
            {"LIKE_NUM": True},
            {"TEXT": {"REGEX": r"[-–—]"}, "OP": "?"},
            {"LIKE_NUM": True, "OP": "?"},
            {"LOWER": {"IN": ["hz", "khz"]}},
        ],
        [
            # Flags NICHT als Feld übergeben, sondern inline (?i) für case-insensitive
            {"TEXT": {"REGEX": r"(?i)^\d+(?:\.\d+)?k?hz$"}}
        ],
    ],
)

# 2) Sibilanz / S-Laute
matcher.add(
    "SIBILANCE",
    [[{"LOWER": {"IN": ["s-laut", "s-laute", "zischelig", "sibilanz", "sibilanzen"]}}]]
)

# 3) Kompressor/Dynamics-Hinweise als Regex auf dem Rohtext
DYN_REGEX = re.compile(r"\b(sidechain|parallel|kompressor|compressor|ratio|threshold|attack|release)\b", re.I)


def tag_post(text: str) -> List[str]:
    """Gibt sortierte Tag-Liste zurück.  Tags: tone_adj, freq, sibilance, dynamics"""
    tags: Set[str] = set()
    doc = nlp(text)

    # Phrase-Matches (Audio-Adjektive)
    if phr(doc):
        tags.add("tone_adj")

    # Token-Matches (Frequenz/Sibilanz)
    for mid, start, end in matcher(doc):
        label = nlp.vocab.strings[mid]
        if label == "FREQ_HZ":
            tags.add("freq")
        elif label == "SIBILANCE":
            tags.add("sibilance")

    # Regex-Dynamics auf dem Rohtext
    if DYN_REGEX.search(text):
        tags.add("dynamics")

    return sorted(tags)


print("spaCy geladen:", nlp.meta.get("name", "blank"), "· Pipeline:", nlp.pipe_names)


spaCy geladen: core_news_sm · Pipeline: ['tok2vec', 'tagger', 'morphologizer', 'parser', 'lemmatizer', 'attribute_ruler', 'ner']


In [2]:
import pandas as pd

examples = [
    "Die Snare ist zu harsch bei 8 kHz, vielleicht De-Esser?",
    "Kick wirkt zu weich – mehr Punch, bitte! 80 Hz boosten?",
    "Vocal nasal & boxig, 800 Hz etwas absenken.",
    "Sidechain vom Kick auf den Bass mit 2–4 dB Ducking.",
    "Gitarren spitz, zwischen 3–4 kHz zu scharf.",
    "Transient-Designer für mehr Attack auf der Snare."
]

rows = [{"text": s, "tags": ", ".join(tag_post(s))} for s in examples]
df = pd.DataFrame(rows)
df

,text,tags
0,"Die Snare ist zu harsch bei 8 kHz, vielleicht ...","freq, tone_adj"
1,"Kick wirkt zu weich – mehr Punch, bitte! 80 Hz...","freq, tone_adj"
2,"Vocal nasal & boxig, 800 Hz etwas absenken.","freq, tone_adj"
3,Sidechain vom Kick auf den Bass mit 2–4 dB Duc...,dynamics
4,"Gitarren spitz, zwischen 3–4 kHz zu scharf.",tone_adj
5,Transient-Designer für mehr Attack auf der Snare.,dynamics


In [3]:
def _assert_has(taglist, tag):
    assert tag in taglist, f"erwarte Tag '{tag}', bekam {taglist}"

# Grundchecks
assert isinstance(tag_post("foo"), list)
_assert_has(tag_post("S-Laute nerven, bitte de-essen!"), "sibilance")
_assert_has(tag_post("800 Hz absenken"), "freq")
_assert_has(tag_post("Sidechain auf Bass"), "dynamics")
print("Smoke-Tests ok ✓")

Smoke-Tests ok ✓


In [4]:
# Speichert einfache Tag-Auswertung unter data/rule_tags.json
out = [{"text": s, "tags": tag_post(s)} for s in examples]
DATA = Path("../data")
DATA.mkdir(parents=True, exist_ok=True)
(Path("../data/rule_tags.json")).write_text(
    json.dumps(out, ensure_ascii=False, indent=2), encoding="utf-8"
)
print("Export geschrieben: ../data/rule_tags.json")

Export geschrieben: ../data/rule_tags.json
